In [21]:
import sys
sys.path.append('On_Multiple_Semantics_for_Declarative_Database_Repairs/')
import On_Multiple_Semantics_for_Declarative_Database_Repairs

from database_generator.dba import DatabaseEngine
from Semantics.end_sem import EndSemantics
from Semantics.stage_sem import StageSemantics
from Semantics.step_sem import StepSemantics
from Semantics.independent_sem import IndependentSemantics

# specify the schema
mas_schema = {"synthetic_data_version_1": ('RAC1P', 'SEX', 'REGION', 'PINCP', 'ST', 'COW', 'CIT', 'NATIVITY', 'MSP', 'SCHL', 'DIS','MIL')
              }


def read_rules(rule_file):
    """read programs from txt file"""
    all_programs = []
    with open(rule_file) as f:
        rules = []
        for line in f:
            if line.strip():
                tbl, r = line.split("|")
                rules.append((tbl, r[:-2]))
            else:
                all_programs.append([r for r in rules])
                rules = []
        all_programs.append(rules)
    return all_programs

# load delta programs
programs = read_rules("On_Multiple_Semantics_for_Declarative_Database_Repairs/../delta_program.txt")

# start the database
db = DatabaseEngine("cr")

tbl_names = ["synthetic_data_version_1"]
subfolder = "/datasets/muse/"

def database_reset(db):
    """reset the database"""
    res = db.execute_query("select current_database();")
    db_name = res[0][0]
    db.delete_tables(tbl_names)
    # db.close_connection()
    db = DatabaseEngine(db_name)
    db.load_database_tables(tbl_names)

# choose a delta program from the programs file
rules = programs[0]
print("delta program:", rules)

# # reset the database between runs
# database_reset(db)

# end_sem = EndSemantics(db, rules, tbl_names)
# end_semantics_result = end_sem.find_mss()
# print("result for end semantics:", end_semantics_result)

# # reset the database between runs
# database_reset(db)

# stage_sem = StageSemantics(db, rules, tbl_names)
# stage_semantics_result = stage_sem.find_mss()
# print("result for stage semantics:", stage_semantics_result)

# reset the database between runs
database_reset(db)

step_sem = StepSemantics(db, rules, tbl_names)
step_semantics_result = step_sem.find_mss(mas_schema)
print("result for step semantics:", step_semantics_result)

# reset the database between runs
# database_reset(db)

# ind_sem = IndependentSemantics(db, rules, tbl_names)
# ind_semantics_result = ind_sem.find_mss(mas_schema)
# print("result for independent semantics:", ind_semantics_result)


delta program: [('synthetic_data_version_1', 'SELECT synthetic_data_version_11.* FROM synthetic_data_version_1 AS synthetic_data_version_11, synthetic_data_version_1 AS synthetic_data_version_12 WHERE synthetic_data_version_11.ST = synthetic_data_version_12.ST AND synthetic_data_version_11.REGION <> synthetic_data_version_12.REGION')]
result for step semantics: {('synthetic_data_version_1', (1, 1, 1, '-5261.019985179817', 18, '1.0', 1, 1, '1.0', '16.0', 2, '4.0')), ('synthetic_data_version_1', (1, 1, 1, '139495.15068842118', 25, '1.0', 1, 1, '6.0', 'None', 2, '4.0')), ('synthetic_data_version_1', (9, 1, 4, 'None', 42, '1.0', 1, 1, 'None', '15.0', 2, '4.0')), ('synthetic_data_version_1', (1, 1, 2, '117982.87449503125', 36, '7.0', 1, 1, '1.0', '15.0', 2, '4.0')), ('synthetic_data_version_1', (1, 1, 2, '-8500.0', 56, 'None', 1, 1, '6.0', 'None', 2, '4.0')), ('synthetic_data_version_1', (1, 2, 3, '50662.513820862754', 51, '1.0', 1, 1, '6.0', '2.0', 2, 'None')), ('synthetic_data_version_1',

In [23]:
len(step_semantics_result)

1306

In [24]:
import pandas as pd
import psycopg2

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(
    database="cr",
    user="postgres",
    password="postgres"
)

# Execute an SQL query and create a dataframe hospital_repaired
muse_data_step = pd.read_sql("SELECT * FROM synthetic_data_version_1", conn).iloc[1:]
# muse_data_step.drop(muse_data_step.index[0])
muse_data_step.columns = muse_data_step.columns.str.upper()

conn.close()
display(muse_data_step)

/Users/kushagraghosh/opt/anaconda3/lib/python3.8/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,RAC1P,SEX,REGION,PINCP,ST,COW,CIT,NATIVITY,MSP,SCHL,DIS,MIL
1,9,2,4,None,12,1.0,4,1,1.0,19.0,2,4.0
2,1,1,4,126359.50607515374,18,1.0,1,1,1.0,19.0,2,4.0
3,1,1,4,75127.85895569899,47,8.0,4,1,1.0,16.0,2,4.0
4,1,2,3,None,9,None,1,1,1.0,19.0,2,4.0
5,9,1,4,10790.54754111301,34,None,1,1,1.0,22.0,2,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,6,1,4,-8500.0,35,1.0,1,1,1.0,21.0,2,4.0
2097,2,2,2,84608.50487677663,6,1.0,1,2,1.0,20.0,2,4.0
2098,1,1,4,13482.011731247945,6,1.0,1,1,5.0,20.0,2,4.0
2099,1,1,4,59692.97071589308,51,None,1,1,4.0,4.0,2,4.0


In [25]:
def make_col_int(df, *args):
    for col in args:
        df[col] = df[col].astype(float)
        df[col] = df[col].astype(int)
def make_col_string(df, *args):
    for col in args:
        df[col] = df[col].astype(str)
        
make_col_int(muse_data_step, 'RAC1P', 'SEX', 'REGION', 'ST', 'CIT', 'NATIVITY', 'DIS')
make_col_string(muse_data_step, 'PINCP', 'COW', 'MSP', 'SCHL', 'MIL')
print(muse_data_step.dtypes)

display(muse_data_step)

RAC1P        int64
SEX          int64
REGION       int64
PINCP       object
ST           int64
COW         object
CIT          int64
NATIVITY     int64
MSP         object
SCHL        object
DIS          int64
MIL         object
dtype: object


,RAC1P,SEX,REGION,PINCP,ST,COW,CIT,NATIVITY,MSP,SCHL,DIS,MIL
1,9,2,4,None,12,1.0,4,1,1.0,19.0,2,4.0
2,1,1,4,126359.50607515374,18,1.0,1,1,1.0,19.0,2,4.0
3,1,1,4,75127.85895569899,47,8.0,4,1,1.0,16.0,2,4.0
4,1,2,3,None,9,None,1,1,1.0,19.0,2,4.0
5,9,1,4,10790.54754111301,34,None,1,1,1.0,22.0,2,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2096,6,1,4,-8500.0,35,1.0,1,1,1.0,21.0,2,4.0
2097,2,2,2,84608.50487677663,6,1.0,1,2,1.0,20.0,2,4.0
2098,1,1,4,13482.011731247945,6,1.0,1,1,5.0,20.0,2,4.0
2099,1,1,4,59692.97071589308,51,None,1,1,4.0,4.0,2,4.0


In [26]:
tuples = [tup[1] for tup in step_semantics_result]
for t in tuples:
    row_index = muse_data_step.index[(muse_data_step == t).all(axis=1)]
    print(row_index)
    muse_data_step = muse_data_step.drop(row_index)

Int64Index([539], dtype='int64')
Int64Index([1652], dtype='int64')
Int64Index([1796], dtype='int64')
Int64Index([2038], dtype='int64')
Int64Index([570], dtype='int64')
Int64Index([1313], dtype='int64')
Int64Index([1088], dtype='int64')
Int64Index([927], dtype='int64')
Int64Index([1690], dtype='int64')
Int64Index([1510], dtype='int64')
Int64Index([1773], dtype='int64')
Int64Index([710], dtype='int64')
Int64Index([1145], dtype='int64')
Int64Index([422], dtype='int64')
Int64Index([1083], dtype='int64')
Int64Index([303], dtype='int64')
Int64Index([928], dtype='int64')
Int64Index([1169], dtype='int64')
Int64Index([362], dtype='int64')
Int64Index([382], dtype='int64')
Int64Index([814], dtype='int64')
Int64Index([1183], dtype='int64')
Int64Index([625], dtype='int64')
Int64Index([2029], dtype='int64')
Int64Index([1703], dtype='int64')
Int64Index([854], dtype='int64')
Int64Index([1357], dtype='int64')
Int64Index([1123], dtype='int64')
Int64Index([544], dtype='int64')
Int64Index([747], dtype='in

Int64Index([1375], dtype='int64')
Int64Index([183], dtype='int64')
Int64Index([594], dtype='int64')
Int64Index([282], dtype='int64')
Int64Index([1323], dtype='int64')
Int64Index([1116], dtype='int64')
Int64Index([985], dtype='int64')
Int64Index([1567], dtype='int64')
Int64Index([133], dtype='int64')
Int64Index([1558], dtype='int64')
Int64Index([1331], dtype='int64')
Int64Index([698], dtype='int64')
Int64Index([257], dtype='int64')
Int64Index([1205], dtype='int64')
Int64Index([430], dtype='int64')
Int64Index([1560], dtype='int64')
Int64Index([384], dtype='int64')
Int64Index([1335], dtype='int64')
Int64Index([501], dtype='int64')
Int64Index([490], dtype='int64')
Int64Index([936], dtype='int64')
Int64Index([483], dtype='int64')
Int64Index([1654], dtype='int64')
Int64Index([1128], dtype='int64')
Int64Index([540], dtype='int64')
Int64Index([858], dtype='int64')
Int64Index([1356], dtype='int64')
Int64Index([1277], dtype='int64')
Int64Index([1581], dtype='int64')
Int64Index([317], dtype='int6

Int64Index([822], dtype='int64')
Int64Index([1951], dtype='int64')
Int64Index([876], dtype='int64')
Int64Index([1721], dtype='int64')
Int64Index([506], dtype='int64')
Int64Index([580], dtype='int64')
Int64Index([1028], dtype='int64')
Int64Index([1058], dtype='int64')
Int64Index([1384], dtype='int64')
Int64Index([2006], dtype='int64')
Int64Index([639], dtype='int64')
Int64Index([1042], dtype='int64')
Int64Index([890], dtype='int64')
Int64Index([428], dtype='int64')
Int64Index([385], dtype='int64')
Int64Index([2098], dtype='int64')
Int64Index([245], dtype='int64')
Int64Index([2041], dtype='int64')
Int64Index([466], dtype='int64')
Int64Index([1794], dtype='int64')
Int64Index([532], dtype='int64')
Int64Index([1483], dtype='int64')
Int64Index([1762], dtype='int64')
Int64Index([190], dtype='int64')
Int64Index([1007], dtype='int64')
Int64Index([1268], dtype='int64')
Int64Index([1154], dtype='int64')
Int64Index([2070], dtype='int64')
Int64Index([365], dtype='int64')
Int64Index([2052], dtype='i

Int64Index([1579], dtype='int64')
Int64Index([1846], dtype='int64')
Int64Index([1899], dtype='int64')
Int64Index([167], dtype='int64')
Int64Index([1705], dtype='int64')
Int64Index([762], dtype='int64')
Int64Index([71], dtype='int64')
Int64Index([1207], dtype='int64')
Int64Index([1857], dtype='int64')
Int64Index([17], dtype='int64')
Int64Index([736], dtype='int64')
Int64Index([1276], dtype='int64')
Int64Index([1609], dtype='int64')
Int64Index([1438], dtype='int64')
Int64Index([63], dtype='int64')
Int64Index([2049], dtype='int64')
Int64Index([549], dtype='int64')
Int64Index([176], dtype='int64')
Int64Index([1640], dtype='int64')
Int64Index([4], dtype='int64')
Int64Index([454], dtype='int64')
Int64Index([1945], dtype='int64')
Int64Index([629], dtype='int64')
Int64Index([155], dtype='int64')
Int64Index([2091], dtype='int64')
Int64Index([678], dtype='int64')
Int64Index([1224], dtype='int64')
Int64Index([1742], dtype='int64')
Int64Index([456], dtype='int64')
Int64Index([1540], dtype='int64')

Int64Index([752], dtype='int64')
Int64Index([1071], dtype='int64')
Int64Index([1695], dtype='int64')
Int64Index([1380], dtype='int64')
Int64Index([1830], dtype='int64')
Int64Index([15], dtype='int64')
Int64Index([2023], dtype='int64')
Int64Index([1534], dtype='int64')
Int64Index([1901], dtype='int64')
Int64Index([2097], dtype='int64')
Int64Index([331], dtype='int64')
Int64Index([799], dtype='int64')
Int64Index([834], dtype='int64')
Int64Index([300], dtype='int64')
Int64Index([1416], dtype='int64')
Int64Index([1897], dtype='int64')
Int64Index([1692], dtype='int64')
Int64Index([1636], dtype='int64')
Int64Index([805], dtype='int64')
Int64Index([1091], dtype='int64')
Int64Index([951], dtype='int64')
Int64Index([1532], dtype='int64')
Int64Index([1896], dtype='int64')
Int64Index([811], dtype='int64')
Int64Index([1506], dtype='int64')
Int64Index([913], dtype='int64')
Int64Index([618], dtype='int64')
Int64Index([2007], dtype='int64')
Int64Index([119], dtype='int64')
Int64Index([27], dtype='int

In [27]:
display(muse_data_step)

,RAC1P,SEX,REGION,PINCP,ST,COW,CIT,NATIVITY,MSP,SCHL,DIS,MIL
2,1,1,4,126359.50607515374,18,1.0,1,1,1.0,19.0,2,4.0
3,1,1,4,75127.85895569899,47,8.0,4,1,1.0,16.0,2,4.0
6,4,2,3,35818.3718385832,12,None,1,1,None,9.0,2,4.0
7,1,1,3,67608.02336629394,36,1.0,1,1,None,14.0,2,2.0
13,1,2,4,-8500.0,45,1.0,1,1,4.0,18.0,2,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2085,1,2,4,85493.60497437986,37,None,1,1,1.0,18.0,2,4.0
2093,2,2,3,91413.82223465937,6,3.0,1,1,1.0,16.0,2,4.0
2096,6,1,4,-8500.0,35,1.0,1,1,1.0,21.0,2,4.0
2099,1,1,4,59692.97071589308,51,None,1,1,4.0,4.0,2,4.0


In [28]:
muse_data_step.to_csv('datasets/muse/muse_data_step.csv',index=False)
muse_data_step[muse_data_step['RAC1P']==7]

,RAC1P,SEX,REGION,PINCP,ST,COW,CIT,NATIVITY,MSP,SCHL,DIS,MIL


In [11]:
from platform import python_version

print(python_version())

3.8.11


In [ ]:
# import glob
# import pandas as pd

# path = 'datasets/muse/full'
# all_files = glob.glob(path + "/*.csv")

# dfs = []
# for filename in all_files:
#     df = pd.read_csv(filename)
#     dfs.append(df)

# combined_df = pd.concat(dfs, ignore_index=True)
# combined_df.to_csv('datasets/muse/muse_data_step.csv',index=False)